# Class Mapping 

|Label | prob | type|
|------|------|-----|
|0     | 0.0  | mono|
|1     | 0.333| mono|
|2     | 0.667| mono|
|3     | 1.0  | mono|
|4     | 0.0  | poly|
|5     | 0.333| poly|
|6     | 0.667| poly|
|7     | 1.0  | poly|

In [1]:
from utils.classifier import *
from utils.dataloader import *
from utils.features import *
from utils.helper import *
from utils.processing import *

In [2]:
RANDOM_STATE = 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
PARAMS = []
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
from sklearn.model_selection import train_test_split
img_path, prob, types, labels= load_data(DATA_PATH)

feature_builder = FeatureExtraction(DATA_PATH, img_path, labels)
X_train, X_test, y_train, y_test = feature_builder.split_data(train_test_split, randome_state=RANDOM_STATE, stratify=True, split_ratio=0.25)

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]


In [4]:
from sklearn.metrics import make_scorer, f1_score, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# autoAug 
# Genearte more 
augment_funcs = [
    flip_x,
    flip_y,
    rotate
]

preprocess_pipeline = [
    (strech_img, {
        }),
    # (clach_img, {
    #     'clipLimit': 2.0,
    #     'tileGridSize': (8, 8)}),
    # (guassian_blur, {
    #     'kernel_size': (0, 0),
    #     'sigmaX': 3}),
    # (lap_feature, {
    #     'dst': -1,
    #     'ksize' : 5}),
    # (morpo_opening, {
    #     'kernel': np.ones((2,2), np.uint8),
    #     'iterations': 1}),
]

# Smaller cell: more features
# 
hog_params = {
    'orient': 10,
    'pix_per_cell': 10,
    'cell_per_block': 1,
    'vis': False,
    'block_norm': 'L2'
}

svm_param = {
    'C': 10,
    'kernel' : 'rbf',
    # 'gamma' : 0.0001,
    # 'class_weight' : 'balanced',
    # 'verbose' : False,
}

grid_svm_params = {
    'estimator': SVC,
    'model_params': {
        'kernel': ['rbf'],
        'C' : [0.1, 0.3, 0.5, 0.7, 1, 3, 5,7 ,9],
        'class_weight': ['balanced']},
    'scoring': {
        "F1_Weighted": make_scorer(f1_score, average='weighted'),
        "Balanced_Accuracy": make_scorer(balanced_accuracy_score),},
    'refit': 'Balanced_Accuracy',
    'verbose' : 3
}

smote_params = {
    'total_classes': np.unique(y_train),
    'grid_param': grid_svm_params,
    'knn' : KNeighborsClassifier,
    'n_neighbors': 5,
}

PARAMS.append(augment_funcs)
PARAMS.append(preprocess_pipeline)
PARAMS.append(hog_params)
# PARAMS.append(svm_param)
# PARAMS.append(grid_svm_params)
PARAMS.append(smote_params)

PARAMS

[[<function utils.processing.flip_x(img)>,
  <function utils.processing.flip_y(img)>,
  <function utils.processing.rotate(img)>],
 [(<function utils.processing.strech_img(img) -> numpy.ndarray>, {})],
 {'orient': 10,
  'pix_per_cell': 10,
  'cell_per_block': 1,
  'vis': False,
  'block_norm': 'L2'},
 {'total_classes': array([0, 1, 2, 3, 4, 5, 6, 7]),
  'grid_param': {'estimator': sklearn.svm._classes.SVC,
   'model_params': {'kernel': ['rbf'],
    'C': [0.1, 0.3, 0.5, 0.7, 1, 3, 5, 7, 9],
    'class_weight': ['balanced']},
   'scoring': {'F1_Weighted': make_scorer(f1_score, average=weighted),
    'Balanced_Accuracy': make_scorer(balanced_accuracy_score)},
   'refit': 'Balanced_Accuracy',
   'verbose': 3},
  'knn': sklearn.neighbors._classification.KNeighborsClassifier,
  'n_neighbors': 5}]

In [5]:
X_train_aug, y_train_aug = feature_builder.augmentation(X_train, y_train, augment_funcs=augment_funcs)

X_train_prepro = feature_builder.preprocess(X_train_aug, preprocess_pipeline = preprocess_pipeline)
X_test_prepro = feature_builder.preprocess(X_test, preprocess_pipeline = preprocess_pipeline)

print(X_train_prepro.shape, y_train_aug.shape)
print(X_test_prepro.shape, y_test.shape)

Augmenting images: 100%|██████████| 1968/1968 [00:00<00:00, 87822.01it/s]


Augmenting done, added image: 900


Pre-processing images: 100%|██████████| 656/656 [00:00<00:00, 6638.14it/s]

(2868, 300, 300) (2868,)
(656, 300, 300) (656,)


In [6]:
train_hog_des = feature_builder.get_hog_features(X_train_prepro, **hog_params)
test_hog_des = feature_builder.get_hog_features(X_test_prepro, **hog_params)

print(train_hog_des.shape)
print(test_hog_des.shape)

Calculating descriptors: 100%|██████████| 656/656 [00:05<00:00, 128.12it/s]

(2868, 9000)
(656, 9000)


In [8]:
result, best_clf = grid_search(train_hog_des, y_train_aug, **grid_svm_params)
pred = best_clf.predict(test_hog_des)
acc, conf_mat = get_report(y_test, pred)

print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END C=0.5, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.685, test=0.513) F1_Weighted: (train=0.626, test=0.498) total time= 1.3min
[CV 2/3] END C=0.5, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.672, test=0.514) F1_Weighted: (train=0.605, test=0.497) total time= 1.3min
[CV 1/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.572, test=0.471) F1_Weighted: (train=0.476, test=0.390) total time= 1.4min
[CV 3/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.594, test=0.435) F1_Weighted: (train=0.521, test=0.416) total time= 1.4min
[CV 2/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.571, test=0.491) F1_Weighted: (train=0.492, test=0.422) total time= 1.4min
[CV 3/3] END C=0.1, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.475, test=0.394) F1_Weighted: (train=0.393, test=0.331) total time=

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train_hog_des)
train_X = scaler.transform(train_hog_des)
test_X = scaler.transform(test_hog_des)  

In [11]:
result1, best_clf1 = grid_search(train_X, y_train_aug, **grid_svm_params)
pred1 = best_clf1.predict(test_X)
acc1, conf_mat1 = get_report(y_test, pred1)

print()
print(f'Overall Test Accuracy: {acc1:.2f}')
print(f'Confusion Matrix: \n{conf_mat1}')
print('-'*100)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 2/3] END C=0.5, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.765, test=0.507) F1_Weighted: (train=0.694, test=0.515) total time= 1.4min
[CV 3/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.642, test=0.425) F1_Weighted: (train=0.559, test=0.393) total time= 1.4min
[CV 1/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.655, test=0.452) F1_Weighted: (train=0.561, test=0.415) total time= 1.4min
[CV 1/3] END C=0.5, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.807, test=0.490) F1_Weighted: (train=0.769, test=0.514) total time= 1.4min
[CV 2/3] END C=0.3, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.637, test=0.472) F1_Weighted: (train=0.527, test=0.416) total time= 1.4min
[CV 1/3] END C=0.1, class_weight=balanced, kernel=rbf; Balanced_Accuracy: (train=0.519, test=0.422) F1_Weighted: (train=0.428, test=0.355) total time=

In [ ]:
# results, best_clfs = one_vs_other_up_sampling(train_X, y_train_aug, **smote_params)

# distance, preds = distance_vote(test_X, best_clfs)

# acc, conf_mat = get_report(y_test, preds)

# print()
# print(f'Overall Test Accuracy: {acc:.2f}')
# print(f'Confusion Matrix: \n{conf_mat}')
# print('-'*100)

In [ ]:
# print_misclassifications(y_test, pred, distance)